# Исследование уровня потребительской лояльности для телекоммуникационной компании

Перед телекоммуникационной компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Для визуализации итогов опроса, проведённого компанией, был подготовлен дашборд. Выводы исследования также представлены в презентации.

## Подключение к базе и выгрузка данных:

In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [ ]:
query = """
SELECT user_id,
        lt_day,
        CASE
            WHEN lt_day <= 365 THEN 'True'
            WHEN lt_day > 365 THEN 'False'
        END AS is_new,
    age,
    CASE
        WHEN gender_segment = 0 THEN 'мужчины'
        WHEN gender_segment = 1 THEN 'женщины'
    END AS gender_segment,
    os_name,
    cpe_type_name,
    location.country AS country,
    location.city AS city,
    SUBSTRING(age_segment.title, 4, 10) AS age_segment,
    SUBSTRING(traffic_segment.title, 4, 10) AS traffic_segment,
    SUBSTRING(lifetime_segment.title, 4, 10) AS lifetime_segment,
    nps_score,
    CASE
        WHEN nps_score <= 6 THEN 'критики'
        WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
        WHEN nps_score > 8 THEN 'сторонники'
    END AS nps_group
FROM user
LEFT OUTER JOIN location ON user.location_id = location.location_id
LEFT OUTER JOIN age_segment ON user.age_gr_id = age_segment.age_gr_id
LEFT OUTER JOIN traffic_segment ON user.tr_gr_id = traffic_segment.tr_gr_id
LEFT OUTER JOIN lifetime_segment ON user.lt_gr_id = lifetime_segment.lt_gr_id
WHERE lt_day >= 0;
"""

In [ ]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,женщины,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,False,53.0,мужчины,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,False,57.0,мужчины,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


Сохраним получившуюся таблицу как CSV-файл.

In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Ссылка на дашборд на сайте Tableau Public:

[Дашборд NPS](https://public.tableau.com/views/NPSProject_16738914841330/sheet5?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

## Ссылка на pdf-файл с презентацией:

[Презентация](https://drive.google.com/file/d/1DBrr5ZWGosRGCd3nwF0ViYJrkC9Mr7-x/view?usp=share_link)